# Hexagonal Architecture (Ports and Adapters)

Hexagonal Architecture, also known as Ports and Adapters Architecture, is an architectural pattern used in software design. It aims to create loosely coupled application components that can be easily tested and maintained.

## 1. Introduction

Hexagonal Architecture is an architectural pattern introduced by Alistair Cockburn. Its primary goal is to make the application more modular and adaptable to changes. The pattern divides the application into three main parts:

- **Domain Layer**: The core business logic.
- **Ports**: Interfaces that define the application's use cases.
- **Adapters**: Implementations of the ports that interact with external systems (e.g., databases, APIs).

## 2. Core Concepts

### Domain Layer

The domain layer contains the core business logic and rules of the application. It is independent of any external systems or frameworks.

### Ports

Ports are interfaces that define the entry points to the domain layer. There are two types of ports:
- **Inbound Ports**: These represent the use cases of the application. They are called by external actors (e.g., UI, APIs).
- **Outbound Ports**: These are interfaces through which the domain interacts with external systems (e.g., repositories, third-party services).

### Adapters

Adapters are implementations of the ports. They adapt the external systems to the domain and vice versa.
- **Inbound Adapters**: These handle the communication from external actors to the domain layer (e.g., controllers, UI).
- **Outbound Adapters**: These handle the communication from the domain layer to external systems (e.g., database repositories, API clients).

## 3. Benefits of Hexagonal Architecture

- **Modularity**: Separation of concerns leads to more modular code.
- **Testability**: Core business logic can be tested independently of external systems.
- **Flexibility**: Easy to replace or change external systems without affecting the core logic.
- **Maintainability**: Clear boundaries and responsibilities make the codebase easier to maintain.

## 4. Implementation

### A Banking Application

**Use Case: Money Transfer**
We will implement a simple money transfer service between accounts.

**Domain Model**
The domain model includes the `Account` class as shown earlier.

**Ports**
The ports include the `AccountRepository` and `TransferService` interfaces.

**Adapters**
The adapters include the `InMemoryAccountRepository` and `SimpleTransferService`.

#### Step 1: Define the Domain Layer
Create the core business logic without any dependencies on external systems.

In [1]:
# domain/model.py
class Account:
    def __init__(self, account_id, balance):
        self.account_id = account_id
        self.balance = balance

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        elif amount <= 0:
            raise ValueError("Amount should be greater than 0")
        self.balance -= amount

    def deposit(self, amount):
        self.balance += amount

#### Step 2: Define Ports
Define interfaces for the use cases and external system interactions.

In [2]:
# domain/ports
from abc import ABC, abstractmethod

class AccountRepository(ABC):
    @abstractmethod
    def find_by_id(self, account_id):
        pass

    @abstractmethod
    def save(self, account):
        pass

class TransferService(ABC):
    @abstractmethod
    def transfer(self, from_account_id, to_account_id, amount):
        pass

#### Step 3: Implement Adapters
Implement the ports to interact with external systems.

In [3]:
# adapters/repositories

class InMemoryAccountRepository(AccountRepository):
    def __init__(self):
        self.accounts = {}

    def find_by_id(self, account_id):
        return self.accounts.get(account_id)

    def save(self, account):
        self.accounts[account.account_id] = account

In [4]:
# application/services

class SimpleTransferService(TransferService):
    def __init__(self, account_repository: AccountRepository):
        self.account_repository = account_repository

    def transfer(self, from_account_id, to_account_id, amount):
        from_account = self.account_repository.find_by_id(from_account_id)
        to_account = self.account_repository.find_by_id(to_account_id)
        
        from_account.withdraw(amount)
        to_account.deposit(amount)
        
        self.account_repository.save(from_account)
        self.account_repository.save(to_account)

### Use Case - Money Transfer

In [6]:
account_repository = InMemoryAccountRepository()

# Create accounts
account1 = Account(account_id=1, balance=100)
account2 = Account(account_id=2, balance=50)

account_repository.save(account1)
account_repository.save(account2)

transfer_service = SimpleTransferService(account_repository)
transfer_service.transfer(1, 2, 30)

print(account_repository.find_by_id(1).balance)
print(account_repository.find_by_id(2).balance) 

70
80


## 5. Testing in Hexagonal Architecture


Hexagonal Architecture promotes testability by isolating the core business logic from external dependencies.

In [9]:
# test_transfer_service
import unittest

class TestTransferService(unittest.TestCase):
    def setUp(self):
        self.account_repository = InMemoryAccountRepository()
        self.transfer_service = SimpleTransferService(self.account_repository)
        
        self.account1 = Account(account_id=1, balance=100)
        self.account2 = Account(account_id=2, balance=50)
        
        self.account_repository.save(self.account1)
        self.account_repository.save(self.account2)

    def test_transfer(self):
        self.transfer_service.transfer(1, 2, 30)
        self.assertEqual(self.account_repository.find_by_id(1).balance, 70)
        self.assertEqual(self.account_repository.find_by_id(2).balance, 80)

    def test_transfer_insufficient_funds(self):
        with self.assertRaises(ValueError):
            self.transfer_service.transfer(1, 2, 200)
            
    def test_transfer_invalid_amount(self):
        with self.assertRaises(ValueError):
            self.transfer_service.transfer(1, 2, -20)

# Run the tests and display the results in the notebook
unittest.main(argv=[''], verbosity=2, exit=False)

test_transfer (__main__.TestTransferService.test_transfer) ... ok
test_transfer_insufficient_funds (__main__.TestTransferService.test_transfer_insufficient_funds) ... ok
test_transfer_invalid_amount (__main__.TestTransferService.test_transfer_invalid_amount) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.034s

OK


## 6. Summary and Best Practices

- **Keep the domain layer pure**: Avoid dependencies on external systems.
- **Define clear interfaces (ports)**: Ensure the domain layer is only dependent on these interfaces.
- **Implement adapters**: Use adapters to connect the domain layer to external systems.
- **Focus on testability**: Write tests for your domain logic independently of external systems.